In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import r2_score
from osgeo import gdal

# Read the training data set
X_train = []
y_train = []

for i in range(0, 16):
    dataset_15_landuse = gdal.Open(f'D:/ARCGIS/2015Landuse_{i}.tif')
    band_15_landuse = dataset_15_landuse.GetRasterBand(1)
    data_15_landuse = np.array(band_15_landuse.ReadAsArray())

    dataset_15_gdp = gdal.Open(f'D:/ARCGIS/2015GDP_{i}.tif')
    band_15_gdp = dataset_15_gdp.GetRasterBand(1)
    data_15_gdp = np.array(band_15_gdp.ReadAsArray())

    dataset_15_carbon = gdal.Open(f'D:/ARCGIS/2015CarbonEmission_{i}.tif')
    band_15_carbon = dataset_15_carbon.GetRasterBand(1)
    data_15_carbon = np.array(band_15_carbon.ReadAsArray())

    dataset_15_population = gdal.Open(f'D:/ARCGIS/2015Population_{i}.tif')
    band_15_population = dataset_15_population.GetRasterBand(1)
    data_15_population = np.array(band_15_population.ReadAsArray())

    dataset_20_carbon = gdal.Open(f'D:/ARCGIS/2020CarbonEmission_{i}.tif')
    band_20_carbon = dataset_20_carbon.GetRasterBand(1)
    data_20_carbon = np.array(band_20_carbon.ReadAsArray())

    data_15_landuse[np.isnan(data_15_landuse)] = 0
    data_15_gdp[np.isnan(data_15_gdp)] = 0
    data_15_carbon[np.isnan(data_15_carbon)] = 0
    data_15_population[np.isnan(data_15_population)] = 0
    data_20_carbon[np.isnan(data_20_carbon)] = 0

    X_train.append(np.column_stack((data_15_landuse.flatten(),
                                    data_15_gdp.flatten(),
                                    data_15_carbon.flatten(),
                                    data_15_population.flatten())))
    y_train.append(data_20_carbon.flatten())

# Convert to a numpy array
X_train = np.concatenate(X_train, axis=0)
y_train = np.concatenate(y_train, axis=0)

# Convert to a Pandas DataFrame object
df_train = pd.DataFrame(X_train, columns=['landuse', 'GDP', 'carbon_2015', 'population'])
df_train['carbon_2020'] = y_train

# Separate features and labels
X_train = df_train[['landuse', 'GDP', 'carbon_2015', 'population']]
y_train = df_train['carbon_2020']

# Create the training set data set
train_ds = tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values))
train_ds = train_ds.batch(32)

# Create a model
model = Sequential([
    LSTM(64, activation='relu', input_shape=(4, 1)),
    Dense(64, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='mse')

# Train the model
model.fit(train_ds, epochs=10, verbose=1)

# Read the test data set
X_test = []
y_test = []

for i in range(16, 20):
    dataset_15_landuse = gdal.Open(f'D:/ARCGIS/2015Landuse_{i}.tif')
    band_15_landuse = dataset_15_landuse.GetRasterBand(1)
    data_15_landuse = np.array(band_15_landuse.ReadAsArray())

    dataset_15_gdp = gdal.Open(f'D:/ARCGIS/2015GDP_{i}.tif')
    band_15_gdp = dataset_15_gdp.GetRasterBand(1)
    data_15_gdp = np.array(band_15_gdp.ReadAsArray())

    dataset_15_carbon = gdal.Open(f'D:/ARCGIS/2015CarbonEmission_{i}.tif')
    band_15_carbon = dataset_15_carbon.GetRasterBand(1)
    data_15_carbon = np.array(band_15_carbon.ReadAsArray())

    dataset_15_population = gdal.Open(f'D:/ARCGIS/2015Population_{i}.tif')
    band_15_population = dataset_15_population.GetRasterBand(1)
    data_15_population = np.array(band_15_population.ReadAsArray())

    dataset_20_carbon = gdal.Open(f'D:/ARCGIS/2020CarbonEmission_{i}.tif')
    band_20_carbon = dataset_20_carbon.GetRasterBand(1)
    data_20_carbon = np.array(band_20_carbon.ReadAsArray())

    data_15_landuse[np.isnan(data_15_landuse)] = 0
    data_15_gdp[np.isnan(data_15_gdp)] = 0
    data_15_carbon[np.isnan(data_15_carbon)] = 0
    data_15_population[np.isnan(data_15_population)] = 0
    data_20_carbon[np.isnan(data_20_carbon)] = 0

    X_test.append(np.column_stack((data_15_landuse.flatten(),
                                   data_15_gdp.flatten(),
                                   data_15_carbon.flatten(),
                                   data_15_population.flatten())))
    y_test.append(data_20_carbon.flatten())

# Convert to a numpy array
X_test = np.concatenate(X_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

# Convert to a Pandas DataFrame object
df_test = pd.DataFrame(X_test, columns=['landuse', 'GDP', 'carbon_2015', 'population'])
df_test['carbon_2020'] = y_test

# Separate features and labels
X_test = df_test[['landuse', 'GDP', 'carbon_2015', 'population']]
y_test = df_test['carbon_2020']

# Create the test set data set
test_ds = tf.data.Dataset.from_tensor_slices((X_test.values, y_test.values))
test_ds = test_ds.batch(32)

# Evaluate the model
test_loss = model.evaluate(test_ds)

# Calculate the model accuracy percentage
model_predictions = model.predict(X_test)
accuracy = r2_score(y_test, model_predictions)

# Output the model score and accuracy percentage
print("Model Score:", test_loss)
print("Model accuracy percentage:", accuracy * 100)

Epoch 1/10
45252/45252 [==============================] - 140s 3ms/step - loss: 5206913843200.0000
Epoch 2/10
45252/45252 [==============================] - 137s 3ms/step - loss: 6479983149056.0000
Epoch 3/10
45252/45252 [==============================] - 135s 3ms/step - loss: 5206079176704.0000
Epoch 4/10
45252/45252 [==============================] - 131s 3ms/step - loss: 5170371493888.0000
Epoch 5/10
45252/45252 [==============================] - 133s 3ms/step - loss: 5316691886080.0000
Epoch 6/10
45252/45252 [==============================] - 131s 3ms/step - loss: 5266224971776.0000
Epoch 7/10
45252/45252 [==============================] - 133s 3ms/step - loss: 5791414747136.0000
Epoch 8/10
45252/45252 [==============================] - 137s 3ms/step - loss: 4845833551872.0000
Epoch 9/10
45252/45252 [==============================] - 129s 3ms/step - loss: 4658671648768.0000
Epoch 10/10
4518/4518 [==============================] - 7s 2ms/step
Model Score: 86117982208.0
Model accurac

In [4]:
from skimage.transform import resize
import tifffile
# Read the prediction data and perform preprocessing
dataset_20_landuse = gdal.Open('E:/2020Landuse.tif')
band_20_landuse = dataset_20_landuse.GetRasterBand(1)
data_20_landuse = np.array(band_20_landuse.ReadAsArray())

dataset_20_gdp = gdal.Open('E:/2020GDP.tif')
band_20_gdp = dataset_20_gdp.GetRasterBand(1)
data_20_gdp = np.array(band_20_gdp.ReadAsArray())

dataset_20_carbon = gdal.Open('E:/2020CarbonEmission.tif')
band_20_carbon = dataset_20_carbon.GetRasterBand(1)
data_20_carbon = np.array(band_20_carbon.ReadAsArray())


dataset_20_population = gdal.Open('E:/2020Population.tif')
band_20_population = dataset_20_population.GetRasterBand(1)
data_20_population = np.array(band_20_population.ReadAsArray())

# Data preprocessing and normalization
data_20_landuse[np.isnan(data_20_landuse)] = 0
data_20_gdp[np.isnan(data_20_gdp)] = 0
data_20_carbon[np.isnan(data_20_carbon)] = 0
data_20_population[np.isnan(data_20_population)] = 0

# Stack the data into a two-dimensional array
X_pred = np.column_stack((data_20_landuse.flatten(),
                          data_20_gdp.flatten(),
                          data_20_carbon.flatten(),
                          
                          data_20_population.flatten()))


# Use the model for prediction
y_pred = model.predict(X_pred)

# Organize the prediction results into an array with the same shape as the input region
pred_shape = data_20_landuse.shape
y_pred = y_pred.reshape(pred_shape)


# Create a new TIF image file and write the adjusted prediction results into it
tifffile.imsave("E:/predicted_carbon_2025.tif", y_pred)

37383/37383 [==============================] - 51s 1ms/step


C:\Users\LHT\AppData\Local\Temp\ipykernel_26008\1620935024.py:44: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave("E:/predicted_carbon_2025.tif", y_pred)
